In [21]:
import torch
import yaml
import os

# Humandoid MARL
from Humanoid_MARL.envs.base_env import GymWrapper, VectorGymWrapper
from Humanoid_MARL.utils.torch_utils import save_models, load_models
from Humanoid_MARL.agent.ppo.train_torch import Agent, eval_unroll, get_agent_actions
from Humanoid_MARL.envs.torch_wrapper import TorchWrapper
from IPython.display import HTML, clear_output
from brax.io import html
import jax
from Humanoid_MARL import envs
from Humanoid_MARL.utils.utils import load_reward_config, load_config

In [22]:
env_name = "point_mass"
config = load_config(env_name)
env = envs.create(config['env_name'], auto_reset=True, **config['env_config'])
env = GymWrapper(env, get_jax_state=True)
env = TorchWrapper(env, device=config['train_config']['device'], get_jax_state=True)

model = "models/20240330_234109_ppo_point_mass.pt"
model_path = os.path.join("../", model)
# env warmup
observation = env.reset()
action = torch.zeros(env.action_space.shape[0] * env.num_agents).to(config['train_config']['device'])
env.step(action)
agents = load_models(model_path, Agent, device=config['train_config']['device'])
jax_states = []
num_steps = 1000

eval_reward = 0.0
episodes = torch.zeros((), device = config['train_config']['device'])
for i in range(num_steps):
    print(f"{i} / {num_steps}")
    logits, action = get_agent_actions(agents, observation, env.obs_dims)
    if config['agent_config'].get("freeze_idx"):
        action[:,config['agent_config'].get("freeze_idx") * 8:(config['agent_config'].get("freeze_idx") * 8) + 8] =  torch.ones_like(action[:,config['agent_config'].get("freeze_idx") * 8:(config['agent_config'].get("freeze_idx") * 8) + 8]) * 0
    jax_state, observation, reward, done, info = env.step(Agent.dist_postprocess(action[0]))
    episodes += torch.sum(done)
    jax_states.append(jax_state)
    print(f"{i} | {info} | DONE [{done}] | Reward [{reward}]")
    eval_reward += reward
    if done:
        observation = env.reset()
        print(f"Episode Done")
        print(f"Total Reward | {eval_reward / episodes}")
print(f"Total Reward | {eval_reward / episodes}")

Models loaded from ../models/20240330_234109_ppo_point_mass.pt
0 / 1000
0 | {'first_obs': tensor([0.0000, 0.0000, 0.2560, 2.0000, 1.0000, 0.2560, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 2.0000, 1.0000, 0.2560, 0.0000, 0.0000, 0.2560],
       device='cuda:0'), 'first_pipeline_state': None, 'reward_chase': tensor([-2.2361,  2.2361], device='cuda:0'), 'reward_forward': tensor([0., 0.], device='cuda:0'), 'reward_tag': tensor([0., -0.], device='cuda:0'), 'steps': tensor(2., device='cuda:0'), 'truncation': tensor(0., device='cuda:0'), 'x_position': tensor([0.0103, 2.0103], device='cuda:0'), 'x_velocity': tensor([0.2069, 0.2069], device='cuda:0'), 'y_position': tensor([0.0103, 1.0103], device='cuda:0'), 'y_velocity': tensor([0.0103, 0.0103], device='cuda:0')} | DONE [0.0] | Reward [tensor([-2.2361,  2.2361], device='cuda:0')]
1 / 1000
1 | {'first_obs': tensor([0.0000, 0.0000, 0.2560, 2.0000, 1.0000, 0.2560, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 2.0000, 1.0000

In [23]:
HTML(html.render(env.sys, [jax_state.pipeline_state for jax_state in jax_states])) 

In [24]:
HTML(html.render(env.sys, [jax_state.pipeline_state for jax_state in jax_states])) 